# Extrapolation

This tutorial explains the calling conventions and various variants that can be used for extrapolation.

In [ ]:
from spatiallyAdaptiveSingleDimension2 import *
from GridOperation import *

# Settings
dim = 2
a = np.zeros(dim)
b = np.ones(dim)
max_tol = 10 ** (-5)
max_evaluations = 10 ** 4

coeffs = np.array([np.float64(i) for i in range(1, dim + 1)])
midpoint = np.ones(dim) * 0.99
f = GenzGaussian(coeffs, midpoint)

# plot function
f.plot(np.ones(dim)*a,np.ones(dim)*b)
reference_solution = f.getAnalyticSolutionIntegral(a, b)
errorOperator = ErrorCalculatorSingleDimVolumeGuided()

## Global Romberg Grid

A Romberg grid consists of (multiple) containers that each contain some slices.
If a container contains only one slice then the slice is extrapolated separately according to the specified option.
Otherwise the whole container is extrapolated using the method provided as an parameter.

Each Romberg grid has three parameters that determine the type of extrapolation: 
* `slice_grouping`: This option determines how grid sliced should be grouped into larger containers.
    * `UNIT`: Each slice has it's own container.
    * `GROUPED`: Slices are grouped into containers that contain a multiple of 2 slices.
    * `GROUPED_OPTIMIZED`: Slices are grouped into containers that contain a multiple of 2 slices. This method also tries to maximize each containers size.
* slice_version: This option determines the extrapolation type of unit slices.
    * `ROMBERG_DEFAULT`: sliced Romberg extrapolation.
    * `TRAPEZOID`: default trapezoidal rule without extrapolation.
    * `ROMBERG_DEFAULT_CONST_SUBTRACTION`: sliced Romberg extrapolation with subtraction of extrapolation constants.
* container_version: This options determines the container type.
    * `ROMBERG_DEFAULT`: executes a default Romberg method inside this container.
    * `LAGRANGE_ROMBERG`: executes a default Romberg method inside this container while missing points are interpolated.
    * `LAGRANGE_FULL_GRID_ROMBERG`: the whole grid is understood as one big container. All missing points up to the maximal level are interpolated. Afterwards a default Romberg method is executed.
    * `SIMPSON_ROMBERG`: Instead of using trapezoidal rules as a base rule, here, the Simpson rule is used.
    

In [ ]:
grid = GlobalRombergGrid(a=a, b=b, modified_basis=False, boundary=True,
                         slice_grouping=SliceGrouping.UNIT,
                         slice_version=SliceVersion.ROMBERG_DEFAULT,
                         container_version=SliceContainerVersion.ROMBERG_DEFAULT)


Afterwards we create the grid operation and spatial adaptive object.
Another option that can be se is `force_balanced_refinement_tree`. 
If enabled, each the refinement tree of each one-dimensional grid stripe is force to a balanced refinement tree.
This means that each node either has zero or two children. 

In [ ]:
balanced = False

operation = Integration(f=f, grid=grid, dim=dim, reference_solution=reference_solution)
adaptiveCombiInstanceSingleDim = SpatiallyAdaptiveSingleDimensions2(a, b, operation=operation, rebalancing=False,
                                                                    force_balanced_refinement_tree=balanced)

Finally we perform the spatially adaptive refinement:

In [ ]:
adaptiveCombiInstanceSingleDim.performSpatiallyAdaptiv(1, 2, errorOperator,
                                                       max_tol, max_evaluations=max_evaluations,
                                                       do_plot=True)

print("Number of points used in refinement:", adaptiveCombiInstanceSingleDim.get_total_num_points())
